# Summary
Cleaning a dataset containing satellites and their parameters. Ultimately, going to combine with other sources to get a larger set of data.

In [1]:
# imports
import pandas as pd

In [2]:
# read excel
sat_df = pd.read_excel('data/UCS-Satellite-Database-1-1-2022.xls')

In [3]:
# look at it
sat_df.head()

,"Name of Satellite, Alternate Names",Current Official Name of Satellite,Country/Org of UN Registry,Country of Operator/Owner,Operator/Owner,Users,Purpose,Detailed Purpose,Class of Orbit,Type of Orbit,...,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66
0,1HOPSAT-TD (1st-generation High Optical Perfor...,1HOPSAT-TD,NR,USA,Hera Systems,Commercial,Earth Observation,Infrared Imaging,LEO,Non-Polar Inclined,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Aalto-1,Aalto-1,Finland,Finland,Aalto University,Civil,Technology Development,NaN,LEO,Sun-Synchronous,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AAUSat-4,AAUSat-4,Denmark,Denmark,University of Aalborg,Civil,Earth Observation,Automatic Identification System (AIS),LEO,Sun-Synchronous,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"ABS-2 (Koreasat-8, ST-3)",ABS-2,NR,Multinational,Asia Broadcast Satellite Ltd.,Commercial,Communications,NaN,GEO,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ABS-2A,ABS-2A,NR,Multinational,Asia Broadcast Satellite Ltd.,Commercial,Communications,NaN,GEO,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Step 1 - Unwanted Columns

Some columns I don't want for the simple reason that they have information I don't care about. I mostly care about the parameters, characteristics, and names of these satellites. Other information might not be needed. Let's take a look:

In [4]:
# look at row names, do we need all of these?
for column in sat_df:
    print(column)

Name of Satellite, Alternate Names
Current Official Name of Satellite
Country/Org of UN Registry
Country of Operator/Owner
Operator/Owner
Users
Purpose
Detailed Purpose
Class of Orbit
Type of Orbit
Longitude of GEO (degrees)
Perigee (km)
Apogee (km)
Eccentricity
Inclination (degrees)
Period (minutes)
Launch Mass (kg.)
Dry Mass (kg.)
Power (watts)
Date of Launch
Expected Lifetime (yrs.)
Contractor
Country of Contractor
Launch Site
Launch Vehicle
COSPAR Number
NORAD Number
Comments
Unnamed: 28
Source Used for Orbital Data
Source
Source.1
Source.2
Source.3
Source.4
Source.5
Source.6
Unnamed: 37
Unnamed: 38
Unnamed: 39
Unnamed: 40
Unnamed: 41
Unnamed: 42
Unnamed: 43
Unnamed: 44
Unnamed: 45
Unnamed: 46
Unnamed: 47
Unnamed: 48
Unnamed: 49
Unnamed: 50
Unnamed: 51
Unnamed: 52
Unnamed: 53
Unnamed: 54
Unnamed: 55
Unnamed: 56
Unnamed: 57
Unnamed: 58
Unnamed: 59
Unnamed: 60
Unnamed: 61
Unnamed: 62
Unnamed: 63
Unnamed: 64
Unnamed: 65
Unnamed: 66


I don't care about the source or any of the unnamed columns. Those will be removed.

In [5]:
# pick the removes
removes = ['Source Used for Orbital Data', 'Source', 'Source.1', 'Source.2', 'Source.3',
          'Source.4', 'Source.5', 'Source.6', 'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39',
           'Unnamed: 40', 'Unnamed: 41', 'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 44',
           'Unnamed: 45', 'Unnamed: 46', 'Unnamed: 47', 'Unnamed: 48', 'Unnamed: 49', 'Unnamed: 50',
           'Unnamed: 51', 'Unnamed: 52', 'Unnamed: 53', 'Unnamed: 54', 'Unnamed: 55', 'Unnamed: 56',
           'Unnamed: 57', 'Unnamed: 58', 'Unnamed: 59', 'Unnamed: 60', 'Unnamed: 61', 'Unnamed: 62',
           'Unnamed: 63', 'Unnamed: 64', 'Unnamed: 65', 'Unnamed: 66', 'Unnamed: 28']
sat_df.drop(columns=removes, inplace=True)

That's a bit better. Let's look at null values next.

# Step 2 -  Incomplete Columns

Some columns may be relevant if they were full, but be so incomplete as to render them useless here. I will look at columns with too much incomplete data. I will then choose a threshold for removal based on how much is missing. Hopefully there will be an easy line to draw here.

In [6]:
# how many null values in each column?
for column in sat_df:
    print(column, len(sat_df[column][sat_df[column].isnull()]))

Name of Satellite, Alternate Names 2
Current Official Name of Satellite 2
Country/Org of UN Registry 3
Country of Operator/Owner 2
Operator/Owner 2
Users 2
Purpose 2
Detailed Purpose 3810
Class of Orbit 2
Type of Orbit 632
Longitude of GEO (degrees) 2
Perigee (km) 2
Apogee (km) 2
Eccentricity 2
Inclination (degrees) 2
Period (minutes) 17
Launch Mass (kg.) 242
Dry Mass (kg.) 4401
Power (watts) 4262
Date of Launch 2
Expected Lifetime (yrs.) 1759
Contractor 4
Country of Contractor 4
Launch Site 3
Launch Vehicle 2
COSPAR Number 2
NORAD Number 2
Comments 3336


It looks like some columns have a LOT of null values, and some have very few. This makes it easy to pick a cut off. 100 will easily take care of the big ones while not touching the ones with few null values.

In [7]:
# remove columns if too many null values:
for column in sat_df:
    if len(sat_df[column][sat_df[column].isnull()]) > 100:
        sat_df.drop(columns=column, inplace=True)

# Step 3 - Rows

We now need to deal with rows with null values. It's possible we might be able to come up with estimates for particular parameters. However, we can already see that there are so few null values compared to the number of satellites as a whole that if we were to straight up remove them, we would still have a similar sized data set. I'm just going to remove the rows with a null value.

In [8]:
# drop any rows with null values - we won't miss them
sat_df.dropna(inplace=True)
len(sat_df)

4832

The length is not much different. And now we have no null values!

# Step 4 - New Parameter

I would like to add a new column for the orbital parameter semi-major axis. This refers to the size of the orbit. It is already implied through other parameters listed here, but I wanted to list it explicitly. It will involve very simple arithmetic.

In [9]:
# semi-major axis
sat_df['Semi-Major Axis (km)'] = (sat_df['Perigee (km)'] + sat_df['Apogee (km)']) / 2

# Step 5 - Sort by NORAD Number

The NORAD Number is an identifier used for the sattelites. There's no great reason to sort it by that, but we can do it anyway. It sort of is like sorting by age, because they are cataloged in chronological order.

In [10]:
sat_df.sort_values('NORAD Number', inplace=True)
sat_df.head()

,"Name of Satellite, Alternate Names",Current Official Name of Satellite,Country/Org of UN Registry,Country of Operator/Owner,Operator/Owner,Users,Purpose,Class of Orbit,Longitude of GEO (degrees),Perigee (km),...,Inclination (degrees),Period (minutes),Date of Launch,Contractor,Country of Contractor,Launch Site,Launch Vehicle,COSPAR Number,NORAD Number,Semi-Major Axis (km)
77,Amsat-Oscar 7 (AO-7),Amsat-Oscar 7,USA,USA,AMSAT-NA,Civil,Communications,LEO,0,1440.0,...,101.40,114.90,1974-11-15,Amsat-NA,USA,Vandenberg AFB,Delta 2310,1974-089B,7530.0,1449.5
4492,"TDRS-3 (Tracking and Data Relay Satellite, TDR...",TDRS-3,USA,USA,National Aeronautics and Space Administration ...,Government,Communications,GEO,-49,35693.0,...,11.53,1436.06,1988-09-29,TRW Defense and Space Systems Group,USA,Cape Canaveral,Space Shuttle (STS 26),1988-091B,19548.0,35785.5
735,FLTSATCOM-8 (USA 46),USA 46,USA,USA,US Navy,Military,Communications,GEO,-14.74,35745.0,...,8.97,1436.13,1989-09-25,"TRW, Defense and Space Systems Group",USA,Cape Canaveral,Atlas Centaur,1989-077A,20253.0,35787.0
1018,"Hubble Space Telescope (HST, Space Telescope)",Hubble Space Telescope,USA,ESA/USA,European Space Agency (ESA)/NASA,Government,Space Science,LEO,0,555.0,...,28.50,95.80,1990-04-25,European Space Agency/NASA,International,Cape Canaveral,Space Shuttle (STS 31),1990-037B,20580.0,557.0
2434,Skynet 4C,Skynet 4C,United Kingdom,United Kingdom,Intelsat/Paradigm Secure Communications (wholl...,Military,Communications,GEO,-1.3,35785.0,...,13.60,1436.07,1990-08-30,Astrium,France/UK/Germany,Guiana Space Center,Ariane 44LP,1990-079A,20776.0,35786.0


Here we have it. A data set ready for use.

Export it as a csv

In [11]:
from pathlib import Path  
filepath = Path('data/df_1.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
sat_df.to_csv(filepath)  